In [1]:
import sys
sys.path.insert(0,'../')

In [2]:
import rasterio
from rasterio.plot import show
from cerulean_cloud.cloud_run_orchestrator.clients import *
from cerulean_cloud.cloud_run_orchestrator.handler import from_tiles_get_offset_shape, from_bounds_get_offset_bounds
from cerulean_cloud.titiler_client import TitilerClient
from cerulean_cloud.tiling import TMS as tiler
from cerulean_cloud.tiling import from_base_tiles_create_offset_tiles

In [3]:
TITILER_URL="https://0xshe4bmk8.execute-api.eu-central-1.amazonaws.com/"
INFERENCE_URL="https://cerulean-cloud-staging-cloud-run-offset-tiles-49b-5qkjkyomta-ew.a.run.app"
AUX_INFRA_DISTANCE="gs://ceruleanml/aux_datasets/infra_locations_01_cogeo.tiff"
sceneid="S1A_IW_GRDH_1SDV_20200729T034859_20200729T034924_033664_03E6D3_93EF"

In [4]:
titiler_client = TitilerClient(url=TITILER_URL)
bounds = titiler_client.get_bounds(sceneid)
stats = titiler_client.get_statistics(sceneid, band="vv")
base_tiles = list(tiler.tiles(*bounds, [9], truncate=False))
offset_image_shape = from_tiles_get_offset_shape(base_tiles, scale=2)
offset_tiles_bounds = from_base_tiles_create_offset_tiles(base_tiles)
offset_bounds = from_bounds_get_offset_bounds(offset_tiles_bounds)

In [5]:
client = CloudRunInferenceClient(url=INFERENCE_URL, titiler_client=titiler_client, sceneid=sceneid, offset_bounds=offset_bounds, offset_image_shape=offset_image_shape, aux_datasets=["ship_density", AUX_INFRA_DISTANCE])

TypeError: from_bounds() got an unexpected keyword argument 'heigth'

In [ ]:
client.aux_datasets

In [ ]:
with client.aux_datasets.open() as ds:
    with rasterio.open("aux_ds.tiff", **ds.profile, mode="w") as dst:
        dst.write(ds.read())

In [ ]:
bounds

In [ ]:
offset_image_shape

In [ ]:
offset_bounds

In [ ]:
import supermercado
tiles_np = np.array([(tile.x, tile.y, tile.z) for tile in base_tiles])
tilexmin, tilexmax, tileymin, tileymax = supermercado.super_utils.get_range(
    tiles_np
)

In [ ]:
tilexmin, tilexmax, tileymin, tileymax